In [1]:
import sys 
import numpy as np

In [2]:
from utils import *
from massfunction import *

from tqdm import tqdm, trange
import matplotlib.pyplot as plt
import os
import emcee
import sys
import numpy as np
import dill as pickle
from scipy.interpolate import interp1d, UnivariateSpline, InterpolatedUnivariateSpline


In [3]:
cosmos_f = open('data/cosmo_params.pkl', 'rb')
cosmo_params = pickle.load(cosmos_f) #cosmo_params is a dict
cosmos_f.close()

a_list_fname = '/oak/stanford/orgs/kipac/users/delon/aemulusnu_massfunction/alist.pkl'
a_list_f = open(a_list_fname, 'rb')
a_list = pickle.load(a_list_f) 
a_list_f.close()

In [4]:
boxs = ['Box_n50_0_1400', 'Box_n50_4_1400']

In [5]:
allN = {}
allM = {}
allAux = {}
allMpart = {}
allMassFunctions = {}
allWeightedCov = {}
allInvWeightedCov = {}
allScaleCov = {}

for box in boxs:
    cosmo = cosmo_params[box]
    mass_function = MassFunction(cosmo)

    h = cosmo['H0']/100
    NvM_fname = '/oak/stanford/orgs/kipac/users/delon/aemulusnu_massfunction/'+box+'_NvsM.pkl'
    NvM_f = open(NvM_fname, 'rb')
    NvMs = pickle.load(NvM_f) #NvMs is a dictionary of dictionaries
    NvM_f.close()

    N_data = {}
    M_data = {}
    aux_data = {}


    vol = -1 #Mpc^3/h^3
    Mpart = -1

    for a in tqdm(a_list):
        c_data = NvMs[a]

        Ms = c_data['M'] #units of h^-1 Msolar
        N = c_data['N']
        edge_pairs = c_data['edge_pairs']
        assert(len(Ms) == len(edge_pairs))
        assert(len(Ms) == len(N))


        if(vol==-1):
            vol = c_data['vol']
        assert(vol == c_data['vol'])

        if(Mpart==-1):
            Mpart = c_data['Mpart']
        assert(Mpart == c_data['Mpart'])

        N_data[a] = []
        M_data[a] = []
        aux_data[a] = []
        for N_curr, M_curr, edge_pair in zip(N, Ms, edge_pairs):
            N_data[a] += [N_curr]
            M_data[a] += [M_curr]
        aux_data[a] = {'edge_pairs':edge_pairs}
    
    with open("/oak/stanford/orgs/kipac/users/delon/aemulusnu_massfunction/%s_massfunction.pkl"%(box), "rb") as f:
        tmp = pickle.load(f)
        mass_function.dlnσinvdMs = tmp[0]
        mass_function.Pka = tmp[1]
        
        
    jackknife_covs_fname = '/oak/stanford/orgs/kipac/users/delon/aemulusnu_massfunction/'+box+'_jackknife_covs.pkl'
    jackknife_covs_f = open(jackknife_covs_fname, 'rb')
    jackknife = pickle.load(jackknife_covs_f)
    jackknife_covs_f.close()
    jack_covs = {a:jackknife[a][1] for a in N_data}

    weighted_cov = {a: jack_covs[a] for a in jack_covs}
    inv_weighted_cov = {a:np.linalg.inv(weighted_cov[a]) for a in weighted_cov}  
    scale_cov = {a:np.log(np.linalg.det(weighted_cov[a])) for a in weighted_cov}

    allWeightedCov[box] = weighted_cov
    allInvWeightedCov[box] = inv_weighted_cov
    allScaleCov[box] = scale_cov

    allN[box] = N_data
    allM[box] = M_data
    allAux[box] = aux_data
    allMpart[box] = Mpart
    allMassFunctions[box] = mass_function

100%|██████████| 14/14 [00:00<00:00, 4123.90it/s]


In [6]:
M_numerics = np.logspace(np.log10(100*Mpart), 17, 100)

In [7]:
forScaleX = []
for box in boxs:
    curr_cosmo_params = list(cosmo_params[box].values())
    for a in a_list:
        z = scaleToRedshift(a)
        σ8 = np.sqrt(sigma2(allMassFunctions[box].Pka[a], 8))
        curr_input = curr_cosmo_params + [z*σ8]
        forScaleX += [curr_input]
forScaleX = np.array(forScaleX)
print(forScaleX.shape)

/home/users/delon/aemulusnu_massfunction/utils.py:36: IntegrationWarning: The occurrence of roundoff error is detected, which prevents 
  the requested tolerance from being achieved.  The error may be 
  underestimated.
  res, err = quad(dσ2dk, 0, 20/R, limit=1000)


(28, 9)


In [8]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

scaler.fit(forScaleX)

# # Standardize the data
# X = scaler.transform(X)
# Xlo = scaler.transform(Xlo)

StandardScaler()

In [9]:
def f(param_values, cosmo, zσ8):
    #bias term shape (4,)
    #linear term shape (4, len(cosmo)+1) 
    X = np.array(cosmo+ [zσ8])
    X = scaler.transform([X])[0]+1 #+1 for some numerical stability
    Wi = param_values[:4]
    Wij = param_values[4:]
    Wij = np.reshape(Wij, (4,len(cosmo)+1))
    
    
    term2 = np.einsum('ij,j->i', Wij, X)

    return Wi + np.einsum('ij,j->i', Wij, X)

In [10]:
def log_prior(param_values):
    for box in boxs:
        curr_cosmo_params = list(cosmo_params[box].values())
        for a in a_list:
            z = scaleToRedshift(a)
            σ8 = np.sqrt(sigma2(allMassFunctions[box].Pka[a], 8))
            tinker_params = f(param_values, curr_cosmo_params, z*σ8)
#             print(tinker_params)
            for param in tinker_params:
                if(param < 0 or param > 5):
                    return -np.inf
    return 0

In [11]:
import multiprocessing
from functools import partial

def process_a(a, param_values):
    z = scaleToRedshift(a)
    σ8 = np.sqrt(sigma2(allMassFunctions[box].Pka[a], 8))
    params = dict(zip(['d','e','f','g'], f(param_values, curr_cosmo_params, z*σ8)))

    tinker_eval = [mass_function.tinker(a, M_c, **params)*vol for M_c in M_numerics]
    f_dndlogM = interp1d(M_numerics, tinker_eval, kind='linear', bounds_error=False, fill_value=0.)

    edge_pairs = allAux[box][a]['edge_pairs']
    model_vals = np.array([quad(f_dndlogM, edge_pair[0], edge_pair[1], epsabs=1e-1)[0]
        for edge_pair in edge_pairs
    ])

    residuals =  model_vals - N_data[a]

    inv_weighted_cov = allInvWeightedCov[box][a]
    scale_cov = allScaleCov[box][a]

    log_prob = -0.5 * (len(inv_weighted_cov) * np.log(2 * np.pi) +
                      np.dot(np.dot(residuals.T, inv_weighted_cov), residuals) +
                      scale_cov)

    if not np.isfinite(log_prob): 
        return -np.inf
    else:
        return log_prob

def log_prob(param_values):
    if log_prior(param_values) == -np.inf:
        return -np.inf

    pool = multiprocessing.Pool()  # Specify the number of processes/cores
    process_a_partial = partial(process_a, param_values = param_values) 
    results = pool.map(process_a_partial, a_list)
    pool.close()
    pool.join()

    oup = np.sum(results)
    return oup

def log_likelihood(param_values):
    lp = log_prior(param_values)
    if not np.isfinite(lp):
        return -np.inf
    return lp + log_prob(param_values)

In [12]:
# def log_prob(param_values):   
#     """
#     Calculates the probability of the given tinker parameters 
    
#     Args:
#         param_values (np.ndarray): Input array of shape (number of params).
        
#     Returns:
#         float: Resulting log probability
#     """

#     if(log_prior(param_values) == -np.inf):
#         return -np.inf

#     oup = 0
#     for box in boxs:
#         curr_cosmo_params = list(cosmo_params[box].values())
#         for a in a_list:
#             z = scaleToRedshift(a)
#             σ8 = np.sqrt(sigma2(allMassFunctions[box].Pka[a], 8))
#             params = dict(zip(['d','e','f','g'], f(param_values, curr_cosmo_params, z*σ8)))

#             tinker_eval = [mass_function.tinker(a, M_c, **params)*vol for M_c in M_numerics]
#             f_dndlogM = interp1d(M_numerics, tinker_eval, kind='linear', bounds_error=False, fill_value=0.)
            
#             print(allAux.keys())
#             print(allAux[box].keys())
#             print(allAux[box][a].keys())
#             model_vals = np.array([quad(f_dndlogM, edge_pair[0], edge_pair[1], epsabs=1e-1)[0]
#                 for edge_pair in allAux[box][a]['edge_pairs']
#             ])


#             residuals =  model_vals-N_data[a]
            
#             inv_weighted_cov = allInvWeightedCov[box][a]
#             scale_cov = allScaleCov[box][a]
            
#             log_prob =  -0.5 * (len(inv_weighted_cov)* np.log(2*np.pi) + 
#                                   np.dot(np.dot(residuals.T, inv_weighted_cov), residuals) + 
#                                   scale_cov)

#             if not np.isfinite(log_prob): 
#                 oup += -np.inf
#             else:
#                 oup += log_prob

def log_likelihood(param_values):
    lp = log_prior(param_values)
    if not np.isfinite(lp):
        return -np.inf
    return lp + log_prob(param_values)

In [13]:
guess = np.random.uniform(low=0, high=1, size=(4+4*9,))
while(not np.isfinite(log_prior(guess))):
    guess = np.random.uniform(low=0, high=1, size=(4+4*9,))

In [14]:
log_likelihood(guess)

/home/users/delon/aemulusnu_massfunction/utils.py:36: IntegrationWarning: The occurrence of roundoff error is detected, which prevents 
  the requested tolerance from being achieved.  The error may be 
  underestimated.
  res, err = quad(dσ2dk, 0, 20/R, limit=1000)
/home/users/delon/aemulusnu_massfunction/utils.py:36: IntegrationWarning: The occurrence of roundoff error is detected, which prevents 
  the requested tolerance from being achieved.  The error may be 
  underestimated.
  res, err = quad(dσ2dk, 0, 20/R, limit=1000)
/home/users/delon/aemulusnu_massfunction/utils.py:36: IntegrationWarning: The occurrence of roundoff error is detected, which prevents 
  the requested tolerance from being achieved.  The error may be 
  underestimated.
  res, err = quad(dσ2dk, 0, 20/R, limit=1000)
/home/users/delon/aemulusnu_massfunction/utils.py:36: IntegrationWarning: The occurrence of roundoff error is detected, which prevents 
  the requested tolerance from being achieved.  The error may be 


-1701852142.438458

In [15]:
print(1)

1


In [ ]:
print('Starting ML Fit')
#Start by sampling with a maximum likelihood approach
from scipy import optimize as optimize

nll = lambda *args: -log_likelihood(*args)
result = optimize.minimize(nll, guess, method="Nelder-Mead", options={
    'maxiter': len(guess)*10000
})
print(box)
print(result)
print(result['x'])

Starting ML Fit


/home/users/delon/aemulusnu_massfunction/utils.py:36: IntegrationWarning: The occurrence of roundoff error is detected, which prevents 
  the requested tolerance from being achieved.  The error may be 
  underestimated.
  res, err = quad(dσ2dk, 0, 20/R, limit=1000)
/home/users/delon/aemulusnu_massfunction/utils.py:36: IntegrationWarning: The occurrence of roundoff error is detected, which prevents 
  the requested tolerance from being achieved.  The error may be 
  underestimated.
  res, err = quad(dσ2dk, 0, 20/R, limit=1000)
/home/users/delon/aemulusnu_massfunction/utils.py:36: IntegrationWarning: The occurrence of roundoff error is detected, which prevents 
  the requested tolerance from being achieved.  The error may be 
  underestimated.
  res, err = quad(dσ2dk, 0, 20/R, limit=1000)
/home/users/delon/aemulusnu_massfunction/utils.py:36: IntegrationWarning: The occurrence of roundoff error is detected, which prevents 
  the requested tolerance from being achieved.  The error may be 


/home/users/delon/aemulusnu_massfunction/utils.py:36: IntegrationWarning: The occurrence of roundoff error is detected, which prevents 
  the requested tolerance from being achieved.  The error may be 
  underestimated.
  res, err = quad(dσ2dk, 0, 20/R, limit=1000)
/home/users/delon/aemulusnu_massfunction/utils.py:36: IntegrationWarning: The occurrence of roundoff error is detected, which prevents 
  the requested tolerance from being achieved.  The error may be 
  underestimated.
  res, err = quad(dσ2dk, 0, 20/R, limit=1000)
/home/users/delon/aemulusnu_massfunction/utils.py:36: IntegrationWarning: The occurrence of roundoff error is detected, which prevents 
  the requested tolerance from being achieved.  The error may be 
  underestimated.
  res, err = quad(dσ2dk, 0, 20/R, limit=1000)
/home/users/delon/aemulusnu_massfunction/utils.py:36: IntegrationWarning: The occurrence of roundoff error is detected, which prevents 
  the requested tolerance from being achieved.  The error may be 


/home/users/delon/aemulusnu_massfunction/utils.py:36: IntegrationWarning: The occurrence of roundoff error is detected, which prevents 
  the requested tolerance from being achieved.  The error may be 
  underestimated.
  res, err = quad(dσ2dk, 0, 20/R, limit=1000)
/home/users/delon/aemulusnu_massfunction/utils.py:36: IntegrationWarning: The occurrence of roundoff error is detected, which prevents 
  the requested tolerance from being achieved.  The error may be 
  underestimated.
  res, err = quad(dσ2dk, 0, 20/R, limit=1000)
/home/users/delon/aemulusnu_massfunction/utils.py:36: IntegrationWarning: The occurrence of roundoff error is detected, which prevents 
  the requested tolerance from being achieved.  The error may be 
  underestimated.
  res, err = quad(dσ2dk, 0, 20/R, limit=1000)
/home/users/delon/aemulusnu_massfunction/utils.py:36: IntegrationWarning: The occurrence of roundoff error is detected, which prevents 
  the requested tolerance from being achieved.  The error may be 


/home/users/delon/aemulusnu_massfunction/utils.py:36: IntegrationWarning: The occurrence of roundoff error is detected, which prevents 
  the requested tolerance from being achieved.  The error may be 
  underestimated.
  res, err = quad(dσ2dk, 0, 20/R, limit=1000)
/home/users/delon/aemulusnu_massfunction/utils.py:36: IntegrationWarning: The occurrence of roundoff error is detected, which prevents 
  the requested tolerance from being achieved.  The error may be 
  underestimated.
  res, err = quad(dσ2dk, 0, 20/R, limit=1000)
/home/users/delon/aemulusnu_massfunction/utils.py:36: IntegrationWarning: The occurrence of roundoff error is detected, which prevents 
  the requested tolerance from being achieved.  The error may be 
  underestimated.
  res, err = quad(dσ2dk, 0, 20/R, limit=1000)
/home/users/delon/aemulusnu_massfunction/utils.py:36: IntegrationWarning: The occurrence of roundoff error is detected, which prevents 
  the requested tolerance from being achieved.  The error may be 


...

/home/users/delon/aemulusnu_massfunction/utils.py:36: IntegrationWarning: The occurrence of roundoff error is detected, which prevents 
  the requested tolerance from being achieved.  The error may be 
  underestimated.
  res, err = quad(dσ2dk, 0, 20/R, limit=1000)
/home/users/delon/aemulusnu_massfunction/utils.py:36: IntegrationWarning: The occurrence of roundoff error is detected, which prevents 
  the requested tolerance from being achieved.  The error may be 
  underestimated.
  res, err = quad(dσ2dk, 0, 20/R, limit=1000)
/home/users/delon/aemulusnu_massfunction/utils.py:36: IntegrationWarning: The occurrence of roundoff error is detected, which prevents 
  the requested tolerance from being achieved.  The error may be 
  underestimated.
  res, err = quad(dσ2dk, 0, 20/R, limit=1000)
/home/users/delon/aemulusnu_massfunction/utils.py:36: IntegrationWarning: The occurrence of roundoff error is detected, which prevents 
  the requested tolerance from being achieved.  The error may be 


/home/users/delon/aemulusnu_massfunction/utils.py:36: IntegrationWarning: The occurrence of roundoff error is detected, which prevents 
  the requested tolerance from being achieved.  The error may be 
  underestimated.
  res, err = quad(dσ2dk, 0, 20/R, limit=1000)
/home/users/delon/aemulusnu_massfunction/utils.py:36: IntegrationWarning: The occurrence of roundoff error is detected, which prevents 
  the requested tolerance from being achieved.  The error may be 
  underestimated.
  res, err = quad(dσ2dk, 0, 20/R, limit=1000)
/home/users/delon/aemulusnu_massfunction/utils.py:36: IntegrationWarning: The occurrence of roundoff error is detected, which prevents 
  the requested tolerance from being achieved.  The error may be 
  underestimated.
  res, err = quad(dσ2dk, 0, 20/R, limit=1000)
/home/users/delon/aemulusnu_massfunction/utils.py:36: IntegrationWarning: The occurrence of roundoff error is detected, which prevents 
  the requested tolerance from being achieved.  The error may be 


/home/users/delon/aemulusnu_massfunction/utils.py:36: IntegrationWarning: The occurrence of roundoff error is detected, which prevents 
  the requested tolerance from being achieved.  The error may be 
  underestimated.
  res, err = quad(dσ2dk, 0, 20/R, limit=1000)
/home/users/delon/aemulusnu_massfunction/utils.py:36: IntegrationWarning: The occurrence of roundoff error is detected, which prevents 
  the requested tolerance from being achieved.  The error may be 
  underestimated.
  res, err = quad(dσ2dk, 0, 20/R, limit=1000)
/home/users/delon/aemulusnu_massfunction/utils.py:36: IntegrationWarning: The occurrence of roundoff error is detected, which prevents 
  the requested tolerance from being achieved.  The error may be 
  underestimated.
  res, err = quad(dσ2dk, 0, 20/R, limit=1000)
/home/users/delon/aemulusnu_massfunction/utils.py:36: IntegrationWarning: The occurrence of roundoff error is detected, which prevents 
  the requested tolerance from being achieved.  The error may be 


/home/users/delon/aemulusnu_massfunction/utils.py:36: IntegrationWarning: The occurrence of roundoff error is detected, which prevents 
  the requested tolerance from being achieved.  The error may be 
  underestimated.
  res, err = quad(dσ2dk, 0, 20/R, limit=1000)
/home/users/delon/aemulusnu_massfunction/utils.py:36: IntegrationWarning: The occurrence of roundoff error is detected, which prevents 
  the requested tolerance from being achieved.  The error may be 
  underestimated.
  res, err = quad(dσ2dk, 0, 20/R, limit=1000)
/home/users/delon/aemulusnu_massfunction/utils.py:36: IntegrationWarning: The occurrence of roundoff error is detected, which prevents 
  the requested tolerance from being achieved.  The error may be 
  underestimated.
  res, err = quad(dσ2dk, 0, 20/R, limit=1000)
/home/users/delon/aemulusnu_massfunction/utils.py:36: IntegrationWarning: The occurrence of roundoff error is detected, which prevents 
  the requested tolerance from being achieved.  The error may be 


/home/users/delon/aemulusnu_massfunction/utils.py:36: IntegrationWarning: The occurrence of roundoff error is detected, which prevents 
  the requested tolerance from being achieved.  The error may be 
  underestimated.
  res, err = quad(dσ2dk, 0, 20/R, limit=1000)
/home/users/delon/aemulusnu_massfunction/utils.py:36: IntegrationWarning: The occurrence of roundoff error is detected, which prevents 
  the requested tolerance from being achieved.  The error may be 
  underestimated.
  res, err = quad(dσ2dk, 0, 20/R, limit=1000)
/home/users/delon/aemulusnu_massfunction/utils.py:36: IntegrationWarning: The occurrence of roundoff error is detected, which prevents 
  the requested tolerance from being achieved.  The error may be 
  underestimated.
  res, err = quad(dσ2dk, 0, 20/R, limit=1000)
/home/users/delon/aemulusnu_massfunction/utils.py:36: IntegrationWarning: The occurrence of roundoff error is detected, which prevents 
  the requested tolerance from being achieved.  The error may be 
